# Processing

## Preparation

In [1]:
import numpy as np
from collections import namedtuple
from src.decorators import benchmark
from src.structures import DefaultOrderedDict

## Spatial harmonization
Workflow
- initial
    - select forest (class value 20) from dataset gl30 - 2010
    - recode values to binary format 20 = 1, 0 = 0
    - select forest (class value 0 - 100) from hansen tree cover 2000
    - recode values to binary format 1 - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
- looping
    - select forest (0 + 10) - 100 from hansen tree cover 2000
    - recode values to binary format (0 + 10) - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
    - do till 30 or Jaccard Index is max
Potential Images
- world agreement map with different 
    - compare chen and hansen treccover in one image
    - sum of both dataset
    - 2 = agreement, 1 = disagreement

In [2]:
# TODO should it work inplace? Benchmark memory occupation! 
def binary_jaccard(arr1, arr2, return_matrix=False):
    """
    Calculates the Jaccard Index (JI) of two equal sized binary arrays or vectors.
    If return_matrix is set to true the method provides the JI and the necessary 
    calculation matrix as a named tuple.
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0). 
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix. 
    :return: float OR (float, namedtuple)
        Defaultly, the method returns only the JI if, return_matrix is set to true the 
        method returns the JI and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = not required, set to 0
    """
    A, B = np.array(arr1, dtype=np.int8), np.array(arr2, dtype=np.int8)
    
    if np.sum(np.logical_or(A<0,A>1)) != 0 or np.sum(np.logical_or(B<0,B>1)) != 0:
        raise ValueError('Attributes should contain only binary attributes!')
  
    C = A + B
    a = (B - C) + B
    b = (A - C) + A
    
    # Total number of attributes where A == 1 and B == 1
    m11 = np.sum(C==2)
    # Total number of attributes where A == 1 and B == 0
    m10 = np.sum(a==-1)
    # Total number of attributes where A == 0 and B == 1
    m01 = np.sum(b==-1)
    
    jaccard = m11 / (m10 + m01 + m11)
    
    if return_matrix:
        Matrix = namedtuple('Matrix', 'm11 m10 m01 m00')
        return jaccard, Matrix(m11, m10, m01, 0)
    return jaccard

# TODO should it work inplace? Benchmark memory occupation! 
def simple_matching_coefficient(arr1, arr2, return_matrix=False):
    """
    Calculates the Simple Matching Coefficient (SMC) of two equal sized arrays or vectors.
    If return_matrix is set to true the method provides the SMC and the necessary calculation 
    matrix as a named tuple.
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0).
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix.
    :return: float OR (float, namedtuple)
        Defaultly, the method returns only the SMC, if return_matrix is
        set to true the method returns the SMC and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = total number of attributes where arr1 == 0 and arr2 == 0
    """
    _, matrix = binary_jaccard(arr1, arr2, True)
    A = np.array(arr1, dtype=np.int8)
    
    # Total number of attributes where A == 0 and B == 0
    m00 = A.size - sum(matrix)
    
    smc = (matrix.m11 + m00) / A.size

    if return_matrix:
        matrix = matrix._replace(m00=m00)
        return smc, matrix
    return smc